#### First look at the data

In [1]:
import pandas as pd

In [2]:
import json
rows = []
for l in open('amazon_music/Digital_Music_5.json'):
    rows.append(json.loads(l))

df = pd.DataFrame(rows)
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/amazon_music/Digital_Music_5.json'

In [ ]:
df

In [ ]:
df['overall'].value_counts()

In [ ]:
df['reviewerID'].value_counts().min(), df['reviewerID'].value_counts().max()

This dataset is meant to be pre-filtered to only include items/users with atleast 5 repeats, yet it seems a little mistakes slipped through. 
We'll enforce the filters ourselves.

In [ ]:
df['asin'].value_counts().min(), df['asin'].value_counts().max()

In [ ]:
df['asin'].value_counts()

In [ ]:
item_counts = df['asin'].value_counts()
user_counts = df['reviewerID'].value_counts()
items_to_remove = item_counts[item_counts < 5].index.tolist()
users_to_remove = user_counts[user_counts < 5].index.tolist()

filtered_df = df[(~df['asin'].isin(items_to_remove)) & (~df['reviewerID'].isin(users_to_remove))].copy()
print(filtered_df.shape, df.shape)

In [ ]:
# Is it actually legit to do filtering?
filtered_df = df.copy()

# Renaming for ease of matching to ml
filtered_df = filtered_df.rename(columns={'asin':'item_id', 'reviewerID':'user_id', 'overall':'rating'})

#### Moving to 0-based indexing

It would be easiest if both user ids and item ids were 0-indexed (e.g. for embedding layers later on, etc.)

In [ ]:
user2idx = dict(zip(filtered_df['user_id'].unique().tolist(), range(filtered_df['user_id'].nunique())))

In [ ]:
item2idx = dict(zip(filtered_df['item_id'].unique().tolist(), range(filtered_df['item_id'].nunique())))

In [ ]:
filtered_df['user_id'] = filtered_df['user_id'].apply(user2idx.get)
filtered_df['item_id'] = filtered_df['item_id'].apply(item2idx.get)

In [ ]:
filtered_df.head(10)

In [ ]:
filtered_df[['user_id', 'item_id']].isnull().max()

In [ ]:
filtered_df['user_id'].max(), filtered_df['user_id'].nunique()

In [ ]:
filtered_df['item_id'].max(), filtered_df['item_id'].nunique()

#### Saving preprocessed dataset and preprocessing metadata

In [ ]:
filtered_df[['user_id', 'item_id', 'rating']].to_csv('data/am_preprocessed.csv', index=False)
json.dump({'user2idx':user2idx, 'item2idx':item2idx}, open('data/am_preprocessing_metadata.json', 'w'))